In [1]:
import os
import sys
import shutil
import numpy as np
import torch
import utils
import vis_utils_layout as vis_utils
from data_layout import LayoutDataset, Tree
import model_layout as model
import tqdm

sys.setrecursionlimit(5000) 

num_gen = 300
num_recon = 200
exp = 'magazine_2.5K'
category = 'magazine'
checkpoint = ''
device = 'cuda:0'

path = '/home/weiran/Projects/RvNN-Layout/GT-Layout/' + category + '/logs/' + exp
data_path = '/home/weiran/Projects/RvNN-Layout/data/' + category + '-ours/' + exp

# load train config
conf = torch.load(path + '/conf.pth')

# load object category information
Tree.load_category_info(conf.category)

# set up device
print(f'Using device: {conf.device}')

recon_dir = path + '/recon-test/'
gen_dir = path + '/generation/'

if os.path.exists(recon_dir):
    shutil.rmtree(recon_dir)
os.mkdir(recon_dir)

if os.path.exists(gen_dir):
    shutil.rmtree(gen_dir)
os.mkdir(gen_dir)

# create models
# we disable probabilistic because we do not need to jitter the decoded z during inference
encoder = model.RecursiveEncoder(conf, variational=True, probabilistic=False)
decoder = model.RecursiveDecoder(conf)

# load the pretrained models
print('Loading ckpt pretrained_encoder.pth')
data_to_restore = torch.load(path + '/ckpts/' + checkpoint + 'net_encoder.pth')
encoder.load_state_dict(data_to_restore, strict=True)
print('DONE\n')
print('Loading ckpt pretrained_decoder.pth')
data_to_restore = torch.load(path + '/ckpts/' + checkpoint + 'net_decoder.pth')
decoder.load_state_dict(data_to_restore, strict=True)
print('DONE\n')

# send to device
encoder.to(device)
decoder.to(device)

# set models to evaluation mode
encoder.eval()
decoder.eval()

# read test.txt
with open(data_path + '/test.txt', 'r') as fin:
    data_list = [l.rstrip() for l in fin.readlines()]

Using device: cuda:1
Loading ckpt pretrained_encoder.pth
DONE

Loading ckpt pretrained_decoder.pth
DONE



In [4]:
i = 2

# load the gt data as the input
obj = LayoutDataset.load_object(data_path + '/' + data_list[i] + '.json')
obj.to(device)

# feed through the encoder to get a code z
# root_code = encoder.encode_structure(obj)
root_code_and_kld = encoder.encode_structure(obj)
root_code = root_code_and_kld[:, :conf.feature_size]

# infer through the decoder to get the reconstructed output
# set maximal tree depth to conf.max_tree_depth
obj_arr = decoder.decode_structure(z=root_code, max_depth=conf.max_tree_depth)
obj_arr.get_arrbox()

# print(obj)
obj.get_arrbox()
print(obj)
print(obj_arr)

0 vertical_branch    {1} 1 Box([0.0, 0.0, 0.6777777671813965, 0.9111111164093018])
  ├0 title [LEAF] {2} 7 Box([0.05000000074505806, 0.0, 0.5444444417953491, 0.17777776718139648])
  ├1 stack_branch    {3} 3 Box([0.0, 0.23333331942558289, 0.6777777671813965, 0.6777777671813965])
  |  ├0 horizontal_branch    {4} 2 Box([0.1666666716337204, 0.28333333134651184, 0.3611111044883728, 0.5277777910232544])
  |  |  ├0 vertical_branch    {5} 1 Box([0.1666666716337204, 0.28333333134651184, 0.16111111640930176, 0.5277777910232544])
  |  |  |  ├0 text [LEAF] {7} 5 Box([0.1666666716337204, 0.28333333134651184, 0.16111111640930176, 0.17777779698371887])
  |  |  |  ├1 text [LEAF] {8} 5 Box([0.1666666716337204, 0.4833333492279053, 0.16111111640930176, 0.21111111342906952])
  |  |  |  ├2 text [LEAF] {9} 5 Box([0.1666666716337204, 0.7166666984558105, 0.16111111640930176, 0.09444444626569748])
  |  |  ├1 vertical_branch    {6} 1 Box([0.3500000238418579, 0.36666667461395264, 0.17777778208255768, 0.227777779

In [4]:
# reconstruct shapes
with torch.no_grad():
    for i in tqdm.tqdm(range(num_recon)):
        # load the gt data as the input
        obj = LayoutDataset.load_object(data_path + '/' + data_list[i] + '.json')
        obj.to(device)

        # feed through the encoder to get a code z
        # root_code = encoder.encode_structure(obj)
        root_code_and_kld = encoder.encode_structure(obj)
        root_code = root_code_and_kld[:, :conf.feature_size]

        # infer through the decoder to get the reconstructed output
        # set maximal tree depth to conf.max_tree_depth
        obj_arr = decoder.decode_structure(z=root_code, max_depth=conf.max_tree_depth)
        obj_arr.get_arrbox()

        # output the hierarchy
        with open(os.path.join(recon_dir, data_list[i] + '_GT.txt'), 'w') as fout:
            fout.write(str(obj)+'\n\n')
            
        with open(os.path.join(recon_dir, data_list[i] + '_PRED.txt'), 'w') as fout:
            fout.write(str(obj_arr)+'\n\n')

        # output the assembled box-shape
        vis_utils.draw_partnet_objects([obj], \
            object_names=['GT'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 5), \
            out_fn=os.path.join(recon_dir, data_list[i] + '_GT.png'))
        
        vis_utils.draw_partnet_objects([obj], \
            object_names=['GT'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 5), \
            out_fn=os.path.join(recon_dir, data_list[i] + '_GT.svg'))
        
        vis_utils.draw_partnet_objects([obj_arr], \
            object_names=['PRED'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 5), \
            out_fn=os.path.join(recon_dir, data_list[i] + '_PRED.png'))

        vis_utils.draw_partnet_objects([obj_arr], \
            object_names=['PRED'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 5), \
            out_fn=os.path.join(recon_dir, data_list[i] + '_PRED.svg'))

100%|██████████| 200/200 [01:19<00:00,  2.50it/s]


In [5]:
# generate shapes
with torch.no_grad():
    for i in tqdm.tqdm(range(num_gen)):
        # get a Gaussian noise
        code = torch.randn(1, conf.feature_size).to(device)
        
        # infer through the model to get the generated hierarchy
        # set maximal tree depth to conf.max_tree_depth
        obj_arr = decoder.decode_structure(z=code, max_depth=conf.max_tree_depth)

        obj_arr.get_arrbox()
        
        # output the hierarchy
        with open(os.path.join(gen_dir, 'gen-%03d.txt'%i), 'w') as fout:
            fout.write(str(obj_arr)+'\n\n')

        # output the assembled box-shape
        vis_utils.draw_partnet_objects([obj_arr],\
                object_names=['GENERATION'], \
                figsize=(5, 5), out_fn=os.path.join(gen_dir, 'gen-%03d.png'%i),\
                leafs_only=True,sem_colors_filename='./part_colors_magazine.txt')
        
        vis_utils.draw_partnet_objects([obj_arr], \
            object_names=['GENERATION'], leafs_only=True, \
            sem_colors_filename='./part_colors_magazine.txt', figsize=(5, 5), \
            out_fn=os.path.join(gen_dir, 'gen-%03d.svg'%i))

100%|██████████| 300/300 [01:01<00:00,  4.86it/s]
